# Enterprise health dashboard

## Introduction - what is this for

As part of the Enterprise Health Dashboard relies on publicly available information, this notebook will ingest and process the data into ready-to-use csv files for PowerBI visualization purposes.

Most of this work has already been performed in a Powerquery format, but in order to lighten the computation load and make the process more transparent, we are moving to a Pandas/Jupyter Notebook script.

The following datasets will be consulted:

**GC Service Inventory and Service Performance**: An inventory of Government of Canada services, their associated service standards and performance<br>
https://open.canada.ca/data/en/dataset/3ac0d080-6149-499a-8b06-7ce5f00ec56c

**Departmental Plans and Departmental Results Reports**: Expenditures and Full Time Equivalents (FTE) by Program and by Organization<br>
https://open.canada.ca/data/en/dataset/a35cf382-690c-4221-a971-cf0fd189a46f/resource/64774bc1-c90a-4ae2-a3ac-d9b50673a895


Utilities built and shared specifically for this purpose:<br>
https://github.com/gc-performance/utilities

**Department list**: A list of every organization, department, agency, with their various associated names in order to align to a single numeric ID per department.  
**Program-service id correspondence**: Converting the long-form program names in the service inventory to the program id's from the Departmental Plans, Departmental Results Reports.

### Conventions;

Whenever a 4-digit year represents a fiscal year, the 4-digit year is the calendar year during which the fiscal year **ended**

## Setting up environment

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 100)

import numpy as np
import re

# Service inventory and service standards
si = pd.read_csv("https://open.canada.ca/data/dataset/3ac0d080-6149-499a-8b06-7ce5f00ec56c/resource/3acf79c0-a5f5-4d9a-a30d-fb5ceba4b60a/download/service_inventory_2018-2023.csv")
ss = pd.read_csv("https://open.canada.ca/data/dataset/3ac0d080-6149-499a-8b06-7ce5f00ec56c/resource/272143a7-533e-42a1-b72d-622116474a21/download/service_standards.csv")

# Departmental Plans and Departmental Results Reports (Main estimates part III)
rbpo = pd.read_csv("https://open.canada.ca/data/dataset/a35cf382-690c-4221-a971-cf0fd189a46f/resource/64774bc1-c90a-4ae2-a3ac-d9b50673a895/download/rbpo_rppo_en.csv")

# Public Accounts: Operating costs by core responsibility
op_cost = pd.read_csv("https://donnees-data.tpsgc-pwgsc.gc.ca/ba1/respessentielles-coreresp/respessentielles-coreresp.csv")

# Utility tables
# Department list
org_var = pd.read_csv("https://raw.githubusercontent.com/gc-performance/utilities/master/goc-org-variants.csv").set_index('org_name_variant')

# Program-service id correspondence
serv_prog = pd.read_csv("https://raw.githubusercontent.com/gc-performance/utilities/master/goc-service-program.csv")


# Some manipulations to use later
# Latest service names
latest_service_names = si.loc[si.groupby('service_id')['fiscal_yr'].idxmax(), ['service_id', 'service_name_en', 'service_name_fr']]

# Correspondence table between core responsibilities and program id
core_resp_program = rbpo.loc[:, ['organization_id', 'core_responsibility', 'program_id', 'program_name', 'fy_ef']]


# align fy format to service inventory, tidy up some tables
fy_cleanup = {'FY ': '', '-': '-20', '/':'-'}

rbpo['fy_ef'] = rbpo['fy_ef'].replace(fy_cleanup, regex=True)
rbpo.rename(columns={
    'fy_ef': 'fiscal_yr',
    'core_responsibility': 'core_responsibility_en'}, inplace=True)

core_resp_program['fy_ef'] = core_resp_program['fy_ef'].replace(fy_cleanup, regex=True)
core_resp_program = core_resp_program.rename(columns={'fy_ef': 'fiscal_yr'})

op_cost['FSCL_YR'] = op_cost['FSCL_YR'].replace(fy_cleanup, regex=True)



# get org id into op_cost table
op_cost = op_cost.set_index('DEPT_EN_DESC').join(org_var).reset_index()

op_cost.rename(columns={
    'FSCL_YR': 'fiscal_yr',
    'DEPT_EN_DESC': 'department_name_en',
    'CR_EN_NM': 'core_responsibility_en',
    'OP_ATHRTY_CY_AMT': 'operating_costs',
    'org_id': 'organization_id'}, inplace=True)

# get rid of extra cols
op_cost = op_cost.loc[:, ['fiscal_yr', 'department_name_en','organization_id', 'core_responsibility_en', 'operating_costs']]


# define string cleaner function
def normalize_string(s):
    # Remove all non-alphanumeric characters (special characters and spaces)
    s = re.sub(r'[^A-Za-z0-9]', '', s)
    # Convert to uppercase
    return s.upper()



### Applications for service
Given a service, what is the volume of interactions (applications) by channel and fiscal year?

In [2]:
# Unpivot (i.e. melt) application volume columnns

# list of columns that contain application / interaction volumes
# These also represent the channel through which the interaction took place

app_cols = [
    'telephone_applications', 
    'online_applications', 
    'in_person_applications', 
    'postal_mail_applications', 
    'email_applications', 
    'fax_applications', 
    'other_applications'
]

si_vol = pd.melt(si, id_vars=['fiscal_yr', 'service_id'], value_vars=app_cols, var_name='channel', value_name='volume')

# remove "_applications" from the channel column to get a clean channel name
si_vol['channel'] = si_vol['channel'].str.replace('_applications', '')

# remove 'NaN' values in volume
si_vol = si_vol.dropna(subset=['volume'])

si_vol

,fiscal_yr,service_id,channel,volume
0,2018-2019,1000,telephone,0.0
2,2018-2019,1001,telephone,0.0
3,2019-2020,1001,telephone,0.0
4,2020-2021,1001,telephone,0.0
6,2022-2023,1002,telephone,0.0
...,...,...,...,...
50382,2022-2023,SRV03559,other,46.0
50383,2022-2023,SRV03560,other,29.0
50385,2022-2023,SRV03562,other,44.0
50386,2022-2023,SRV03563,other,1566.0


### Online interaction points
Given a service, which online interaction points are activated?

In [3]:
# Unpivot (i.e. melt) online interaction point columns

# list of columns that represent online interaction point activation
oip_cols = [
    'e_registration', 
    'e_authentication', 
    'e_application', 
    'e_decision', 
    'e_issuance', 
    'e_feedback', 
]

si_oip = pd.melt(si, id_vars=['fiscal_yr', 'service_id'], value_vars=oip_cols, var_name='online_interaction_point', value_name='activation')

# add a column to indicate the sort position of the online interaction point
si_oip['online_interaction_point_sort'] = si_oip['online_interaction_point'].apply(lambda x: oip_cols.index(x)+1)

# remove "e_" from the online interaction point column to get a clean name
si_oip['online_interaction_point'] = si_oip['online_interaction_point'].str.replace('e_', '')

# might dump old years, only take latest year
si_oip = si_oip.loc[si_oip.groupby(['service_id', 'online_interaction_point'])['fiscal_yr'].idxmax()].sort_values(by=['service_id', 'online_interaction_point_sort'])

si_oip

,fiscal_yr,service_id,online_interaction_point,activation,online_interaction_point_sort
3726,2022-2023,1,registration,N,1
10925,2022-2023,1,authentication,N,2
18124,2022-2023,1,application,N,3
25323,2022-2023,1,decision,N,4
32522,2022-2023,1,issuance,N,5
...,...,...,...,...,...
14397,2022-2023,SRV03569,authentication,N,2
21596,2022-2023,SRV03569,application,Y,3
28795,2022-2023,SRV03569,decision,Y,4
35994,2022-2023,SRV03569,issuance,Y,5


### Timeliness service standard performance

Given a service, what is the volume of interactions that met the target vs not, by fiscal year?

In [4]:
# Filter the DataFrame for rows where 'service_std_type' is 'Timeliness', group by 'fiscal_yr' 
# and 'service_id', sum the 'volume_meeting_target' and 'total_volume' columns, and reset the index.

ss_tml_perf_vol = ss.loc[ss['service_std_type'] == 'Timeliness'].groupby(['fiscal_yr', 'service_id'])[['volume_meeting_target', 'total_volume']].sum().reset_index()

ss_tml_perf_vol['volume_not_meeting_target'] = ss_tml_perf_vol['total_volume']-ss_tml_perf_vol['volume_meeting_target']

ss_tml_perf_vol

,fiscal_yr,service_id,volume_meeting_target,total_volume,volume_not_meeting_target
0,2018-2019,1,26441.0,28169.0,1728.0
1,2018-2019,10,15313.0,19701.0,4388.0
2,2018-2019,1001,355989.0,387297.0,31308.0
3,2018-2019,1002,0.0,0.0,0.0
4,2018-2019,1003,2277975.0,2853271.0,575296.0
...,...,...,...,...,...
3813,2022-2023,SRV03564,45.0,78.0,33.0
3814,2022-2023,SRV03565,27.0,27.0,0.0
3815,2022-2023,SRV03566,16.0,20.0,4.0
3816,2022-2023,SRV03567,630.0,793.0,163.0


### MAF score calculation for Client-centric service design and delivery
Determining the results of MAF scores

References to methodology can be found here
https://www.canada.ca/en/treasury-board-secretariat/services/management-accountability-framework/maf-methodologies/2022-2023-im-it.html#toc-1

#### Question 1: Existence of service standards
As service standards are required under the Policy on Service and Digital, what is the percentage of services that have service standards?

In [5]:
# setting up the score bins and corresponding results for use with pd.cut
score_bins = [0, 50, 80, 101]
score_results = ['low', 'medium', 'high']

In [6]:
maf1 = si.loc[:, ['fiscal_yr', 'service_id', 'department_name_en']]
maf1['service_std_tf'] = si[['fiscal_yr', 'service_id']].isin(ss[['fiscal_yr', 'service_id']].to_dict(orient='list')).all(axis=1)

maf1_num = maf1.groupby(['fiscal_yr', 'department_name_en'])['service_id'].count().reset_index()
maf1_denom = maf1.groupby(['fiscal_yr', 'department_name_en'])['service_std_tf'].sum().reset_index()

maf1 = pd.merge(
    maf1_num,
    maf1_denom,
    on=['fiscal_yr', 'department_name_en'],
    how='left'
).rename(columns={'service_id':'service_count', 'service_std_tf':'service_with_std_count'})

maf1['maf1_score'] = (maf1['service_with_std_count']/maf1['service_count'])*100
maf1['maf1_result'] = pd.cut(maf1['maf1_score'], bins=score_bins, labels=score_results, right=False)

maf1

,fiscal_yr,department_name_en,service_count,service_with_std_count,maf1_score,maf1_result
0,2018-2019,Administrative Tribunals Support Service of Ca...,4,3,75.000000,medium
1,2018-2019,Agriculture and Agri-Food Canada,30,30,100.000000,high
2,2018-2019,Atlantic Canada Opportunities Agency,5,3,60.000000,medium
3,2018-2019,Canada Border Services Agency,42,27,64.285714,medium
4,2018-2019,Canada Economic Development for Quebec Regions,3,3,100.000000,high
...,...,...,...,...,...,...
372,2022-2023,Transportation Safety Board of Canada,4,0,0.000000,low
373,2022-2023,Treasury Board of Canada Secretariat,27,22,81.481481,high
374,2022-2023,Veterans Affairs Canada,30,20,66.666667,medium
375,2022-2023,Veterans Review and Appeal Board,1,1,100.000000,high


#### Question 2: Service standard targets
What is the percentage of service standards that met their target?

In [7]:
maf2 = ss.loc[:, ['fiscal_yr', 'service_std_id', 'department_name_en', 'target_met']].dropna()

maf2_num = maf2[maf2['target_met']=='Y'].groupby(['fiscal_yr', 'department_name_en'])['service_std_id'].count().reset_index()
maf2_denom = maf2.groupby(['fiscal_yr', 'department_name_en'])['service_std_id'].count().reset_index()

maf2 = pd.merge(
    maf2_num,
    maf2_denom,
    suffixes=['_met','_total'],
    on=['fiscal_yr', 'department_name_en'],
    how='left'
)

maf2['maf2_score'] = (maf2['service_std_id_met']/maf2['service_std_id_total'])*100
maf2['maf2_result'] = pd.cut(maf2['maf2_score'], bins=score_bins, labels=score_results, right=False)

maf2

,fiscal_yr,department_name_en,service_std_id_met,service_std_id_total,maf2_score,maf2_result
0,2018-2019,Administrative Tribunals Support Service of Ca...,1,2,50.000000,medium
1,2018-2019,Agriculture and Agri-Food Canada,88,112,78.571429,medium
2,2018-2019,Atlantic Canada Opportunities Agency,4,7,57.142857,medium
3,2018-2019,Canada Border Services Agency,26,43,60.465116,medium
4,2018-2019,Canada Economic Development for Quebec Regions,1,7,14.285714,low
...,...,...,...,...,...,...
274,2022-2023,Statistics Canada,36,37,97.297297,high
275,2022-2023,Transport Canada,112,204,54.901961,medium
276,2022-2023,Treasury Board of Canada Secretariat,29,30,96.666667,high
277,2022-2023,Veterans Affairs Canada,18,24,75.000000,medium


#### Question 3: Real-time performance for service standards

As real-time performance reporting is required under the Directive on Service and Digital, what is the extent to which real-time performance reporting for services is published?

Real-time URL data is unreliable

#### Question 4: Service standards reviews

What is the percentage of service standards which have been reviewed?

GCSS review field is no longer being collected as of 2023-24 dataset

#### Question 5: Online end-to-end
As online end-to-end availability of services is required under the Policy on Service and Digital, what is the percentage of applicable services that can be completed online end-to-end?

In [8]:
oip_cols = [
    'e_registration', 
    'e_authentication', 
    'e_application', 
    'e_decision', 
    'e_issuance', 
    'e_feedback'
]

# Melt the DataFrame
maf5 = pd.melt(si, id_vars=['fiscal_yr', 'service_id', 'department_name_en'], value_vars=oip_cols, var_name='online_interaction_point', value_name='activation')

# Create boolean columns for activation states
maf5['activation_y'] = (maf5['activation'] == 'Y')
maf5['activation_n'] = (maf5['activation'] == 'N')
maf5['activation_nan'] = maf5['activation'].isna()

# Group by and sum the activation columns
maf5 = maf5.groupby(['fiscal_yr', 'department_name_en', 'service_id'])[['activation_y', 'activation_n', 'activation_nan']].sum().reset_index()

# Determine conditions for online_e2e
conditions = [
    (maf5['activation_nan'] == 6),  # All interaction points are NaN
    (maf5['activation_n'] > 0)      # Some interaction points are 'N'
]
choices = [None, False]

maf5['online_e2e'] = np.select(conditions, choices, default=True).astype(bool)

# remove all Nan/Nones
maf5 = maf5.dropna(subset=['online_e2e'])

# Determine department-level counts for online e2e services and all services
maf5 = maf5.groupby(['fiscal_yr', 'department_name_en']).agg(
    online_e2e_count=('online_e2e', 'sum'), # this is wizardry to me... still not sure what is happening
    service_count=('service_id', 'nunique')
).reset_index()

# Determine score and associated result
maf5['maf5_score'] = (maf5['online_e2e_count']/maf5['service_count'])*100
maf5['maf5_result'] = pd.cut(maf5['maf5_score'], bins=score_bins, labels=score_results, right=False)

maf5



,fiscal_yr,department_name_en,online_e2e_count,service_count,maf5_score,maf5_result
0,2018-2019,Administrative Tribunals Support Service of Ca...,0,4,0.000000,low
1,2018-2019,Agriculture and Agri-Food Canada,9,30,30.000000,low
2,2018-2019,Atlantic Canada Opportunities Agency,1,5,20.000000,low
3,2018-2019,Canada Border Services Agency,2,42,4.761905,low
4,2018-2019,Canada Economic Development for Quebec Regions,0,3,0.000000,low
...,...,...,...,...,...,...
372,2022-2023,Transportation Safety Board of Canada,0,4,0.000000,low
373,2022-2023,Treasury Board of Canada Secretariat,19,27,70.370370,medium
374,2022-2023,Veterans Affairs Canada,19,30,63.333333,medium
375,2022-2023,Veterans Review and Appeal Board,0,1,0.000000,low


#### Question 6: Online client interaction points
As online end-to-end availability of services is required under the Policy on Service and Digital, what is the percentage of client interaction points that are available online for services?

In [9]:
oip_cols = [
    'e_registration', 
    'e_authentication', 
    'e_application', 
    'e_decision', 
    'e_issuance', 
    'e_feedback'
]

# Melt the DataFrame
maf6 = pd.melt(si, id_vars=['fiscal_yr', 'service_id', 'department_name_en'], value_vars=oip_cols, var_name='online_interaction_point', value_name='activation').dropna()

maf6['activation'] = (maf6['activation'] == 'Y')

maf6 = maf6.groupby(['fiscal_yr', 'department_name_en']).agg(
    activated_point_count=('activation', 'sum'), # this is wizardry to me... still not sure what is happening
    point_count=('service_id', 'count')
).reset_index()

# Determine score and associated result
maf6['maf6_score'] = (maf6['activated_point_count']/maf6['point_count'])*100
maf6['maf6_result'] = pd.cut(maf6['maf6_score'], bins=score_bins, labels=score_results, right=False)


maf6

,fiscal_yr,department_name_en,activated_point_count,point_count,maf6_score,maf6_result
0,2018-2019,Administrative Tribunals Support Service of Ca...,2,6,33.333333,low
1,2018-2019,Agriculture and Agri-Food Canada,107,159,67.295597,medium
2,2018-2019,Atlantic Canada Opportunities Agency,11,13,84.615385,high
3,2018-2019,Canada Border Services Agency,32,179,17.877095,low
4,2018-2019,Canada Economic Development for Quebec Regions,4,13,30.769231,low
...,...,...,...,...,...,...
359,2022-2023,Transportation Safety Board of Canada,0,24,0.000000,low
360,2022-2023,Treasury Board of Canada Secretariat,105,116,90.517241,high
361,2022-2023,Veterans Affairs Canada,123,141,87.234043,high
362,2022-2023,Veterans Review and Appeal Board,5,6,83.333333,high


#### Question 7: ICT Accessibility
As accessibility is required under the Policy on Service and Digital, what is the percentage of services available online that have been assessed for ICT accessibility?

Accessibility data is garbage, and we are no longer collecting it through the serice inventory

## Combining other datasets with service inventory and service standards

### Spending and FTEs for programs responsible for service delivery

Given a service, what are the number of actual and planned FTEs by fiscal year for the program responsible for service delivery? What is the actual and planned spending?

In [10]:
rbpo.head()

,fiscal_yr,organization_id,organization,core_responsibility_en,program_id,program_name,planned_spending_1,actual_spending,planned_spending_2,planned_spending_3,planned_ftes_1,actual_ftes,planned_ftes_2,planned_ftes_3,planning_explanation,variance_explanation
0,2018-2019,1,Department of Agriculture and Agri-Food,Domestic and International Markets,BWN01,Trade and Market Expansion,53105701.0,52360723.82,53014508.0,53014508.0,171.0,183.0,171.0,171.0,NaN,NaN
1,2018-2019,1,Department of Agriculture and Agri-Food,Domestic and International Markets,BWN02,Sector Engagement and Development,33331249.0,34247456.65,30455570.0,30455570.0,179.0,184.0,179.0,179.0,NaN,NaN
2,2018-2019,1,Department of Agriculture and Agri-Food,Domestic and International Markets,BWN03,Farm Products Council of Canada,3048578.0,2520779.52,3048552.0,3048552.0,23.0,17.0,23.0,23.0,NaN,Actual spending was lower than planned spendin...
3,2018-2019,1,Department of Agriculture and Agri-Food,Domestic and International Markets,BWN04,Dairy Programs,94238832.0,99881679.91,83258832.0,78288832.0,36.0,46.0,37.0,37.0,NaN,Actual full-time equivalents were higher than ...
4,2018-2019,1,Department of Agriculture and Agri-Food,Domestic and International Markets,BWN05,Canadian Pari-Mutuel Agency,0.0,-317141.62,-216000.0,-53000.0,31.0,31.0,31.0,31.0,NaN,The Canadian Pari-Mutuel Agency is not funded ...


In [11]:

# Reformat program data table to be easier to work with, filter out irrelevant information

# Define columns related to measures: spending and FTEs (planned and actual)
fte_spend_cols = [
    'planned_spending_1', 'actual_spending', 'planned_spending_2', 'planned_spending_3',
    'planned_ftes_1', 'actual_ftes', 'planned_ftes_2', 'planned_ftes_3'
]

# Melt (unpivot) the DataFrame to long format
rbpo_melted = pd.melt(
    rbpo, 
    id_vars=['fiscal_yr', 'organization_id', 'program_id', 'core_responsibility_en'], 
    value_vars=fte_spend_cols, 
    var_name='plan_actual_yr', 
    value_name='measure'
)

# Split 'plan_actual_yr' into separate columns for planned/actual, spending/FTEs, and year adjustment
rbpo_melted[['planned_actual', 'spending_fte', 'yr_adjust']] = rbpo_melted['plan_actual_yr'].str.split('_', expand=True)
rbpo_melted['yr_adjust'] = rbpo_melted['yr_adjust'].fillna('1').astype(int) - 1

# Calculate 'measure_yr' and 'report_yr' from 'fiscal_yr' and 'yr_adjust'
rbpo_melted['measure_yr'] = rbpo_melted['fiscal_yr'].str.split('-').str[1].astype(int) + rbpo_melted['yr_adjust']
rbpo_melted['report_yr'] = rbpo_melted['fiscal_yr'].str.split('-').str[1].astype(int)

# Get the latest fiscal year from the Service inventory (four digit fy, year of end of fy)
latest_si_fy = si['fiscal_yr'].str.split('-').str[1].astype(int).max()

# Separate actuals and future planned data (beyond the latest service fiscal year)
rbpo_melted_actuals = rbpo_melted[rbpo_melted['planned_actual'] == 'actual']
rbpo_melted_planned = rbpo_melted[
    (rbpo_melted['planned_actual'] == 'planned') & (rbpo_melted['report_yr'] > latest_si_fy)
]

# Sort and drop duplicate planned entries, keeping the latest by 'report_yr'
rbpo_melted_planned = rbpo_melted_planned.sort_values(
    by=['report_yr', 'organization_id', 'program_id', 'spending_fte'], 
    ascending=False
).drop_duplicates(subset=['measure_yr','organization_id', 'program_id', 'spending_fte'])

# Concatenate actuals and planned entries, drop any remaining NaNs
rbpo_melted = pd.concat([rbpo_melted_planned, rbpo_melted_actuals]).dropna()

# Pivot to get a wide format table with spending/FTE columns, aggregating with 'sum'
rbpo_melted = rbpo_melted.pivot_table(
    index=['organization_id', 'core_responsibility_en', 'program_id', 'report_yr', 'measure_yr', 'planned_actual'], 
    columns=['spending_fte'], 
    values='measure', 
    aggfunc='sum'
).sort_values(
    by=['organization_id', 'program_id', 'report_yr','measure_yr']
).reset_index()

# Set up a fiscal year column  to be able to include years beyond the service inventory when joining.
# if measure year > latest service fy, = latest service fy

rbpo_melted.loc[rbpo_melted['measure_yr']>latest_si_fy, 'si_link_yr'] = latest_si_fy
rbpo_melted.loc[rbpo_melted['measure_yr']<=latest_si_fy, 'si_link_yr'] = rbpo_melted['measure_yr']

rbpo_melted['si_link_yr'] =rbpo_melted['si_link_yr'].astype(int) 

rbpo_melted.head()

spending_fte,organization_id,core_responsibility_en,program_id,report_yr,measure_yr,planned_actual,ftes,spending,si_link_yr
0,1,Domestic and International Markets,BWN01,2019,2019,actual,183.0,52360723.82,2019
1,1,Domestic and International Markets,BWN01,2020,2020,actual,191.0,58764607.74,2020
2,1,Domestic and International Markets,BWN01,2021,2021,actual,190.0,49132118.39,2021
3,1,Domestic and International Markets,BWN01,2022,2022,actual,181.0,47150140.18,2022
4,1,Domestic and International Markets,BWN01,2023,2023,actual,183.0,56134845.18,2023


In [12]:
# Get org_id into service inventory
# Set index for department and organization variant data
temp1 = si.set_index('department_name_en')
temp2 = org_var

# Join on the department name and include org_id, then set new multi-index
temp3 = temp1.join(temp2)[['service_id', 'fiscal_yr', 'org_id']].set_index(['service_id', 'fiscal_yr'])

# Get the program_id into the service inventory
# Set index for service-program correspondence table
temp4 = serv_prog.set_index(['fiscal_yr', 'service_id'])

# Join the service inventory and the program correspondence table then clean up by resetting the index and dropping NaNs
temp5 = temp3.join(temp4).reset_index().dropna()

# Generate a 4-digit year in the service inventory to link to the program data
temp5['si_link_yr'] = temp5['fiscal_yr'].str.split('-').str[1].astype(int)

# Set a new multi-index for the expanded service inventory and rename org_id to align to the program table
temp5 = temp5.rename(columns={'org_id': 'organization_id'}).set_index(['si_link_yr', 'organization_id', 'program_id'])

# Set index for program data and join with expanded service inventory
temp6 = rbpo_melted.set_index(['si_link_yr', 'organization_id', 'program_id'])

service_fte_spending = temp5.join(temp6, lsuffix='_si', rsuffix='_program').reset_index()

service_fte_spending


,si_link_yr,organization_id,program_id,service_id,fiscal_yr,core_responsibility_en,report_yr,measure_yr,planned_actual,ftes,spending
0,2019,129,BWM06,1000,2018-2019,Rights and Self-Determination,2019.0,2019.0,actual,7.0,3.690273e+07
1,2023,128,BGN01,1001,2022-2023,Pensions and Benefits,2023.0,2023.0,actual,3679.0,7.056224e+10
2,2023,128,BGN01,1001,2022-2023,Pensions and Benefits,2024.0,2024.0,planned,3767.0,7.705208e+10
3,2023,128,BGN01,1001,2022-2023,Pensions and Benefits,2025.0,2025.0,planned,3829.0,8.158422e+10
4,2023,128,BGN01,1001,2022-2023,Pensions and Benefits,2025.0,2026.0,planned,2641.0,8.646568e+10
...,...,...,...,...,...,...,...,...,...,...,...
16409,2023,138,ISS02,SRV03563,2022-2023,Internal Services,2023.0,2023.0,actual,126.0,1.572317e+07
16410,2023,138,ISS02,SRV03563,2022-2023,Internal Services,2024.0,2024.0,planned,132.0,1.653416e+07
16411,2023,138,ISS02,SRV03563,2022-2023,Internal Services,2025.0,2025.0,planned,139.0,1.741480e+07
16412,2023,138,ISS02,SRV03563,2022-2023,Internal Services,2025.0,2026.0,planned,124.0,1.538176e+07


In [13]:
# add core responsibility codes and clean name to working table for program data
rbpo_melted['core_resp_id'] = rbpo_melted['program_id'].str[:3] + '00'
rbpo_melted['core_resp_en_clean'] = rbpo_melted['core_responsibility_en'].apply(normalize_string)

rbpo_melted

spending_fte,organization_id,core_responsibility_en,program_id,report_yr,measure_yr,planned_actual,ftes,spending,si_link_yr,core_resp_id,core_resp_en_clean
0,1,Domestic and International Markets,BWN01,2019,2019,actual,183.0,52360723.82,2019,BWN00,DOMESTICANDINTERNATIONALMARKETS
1,1,Domestic and International Markets,BWN01,2020,2020,actual,191.0,58764607.74,2020,BWN00,DOMESTICANDINTERNATIONALMARKETS
2,1,Domestic and International Markets,BWN01,2021,2021,actual,190.0,49132118.39,2021,BWN00,DOMESTICANDINTERNATIONALMARKETS
3,1,Domestic and International Markets,BWN01,2022,2022,actual,181.0,47150140.18,2022,BWN00,DOMESTICANDINTERNATIONALMARKETS
4,1,Domestic and International Markets,BWN01,2023,2023,actual,183.0,56134845.18,2023,BWN00,DOMESTICANDINTERNATIONALMARKETS
...,...,...,...,...,...,...,...,...,...,...,...
10870,561,Internal Services,ISS00,2023,2023,actual,38.0,7067900.98,2023,ISS00,INTERNALSERVICES
10871,561,Internal Services,ISS00,2024,2024,planned,32.0,5077664.00,2023,ISS00,INTERNALSERVICES
10872,561,Internal Services,ISS00,2025,2025,planned,41.0,4512390.00,2023,ISS00,INTERNALSERVICES
10873,561,Internal Services,ISS00,2025,2026,planned,41.0,4581144.00,2023,ISS00,INTERNALSERVICES


In [15]:
#determine total FTEs by program, org, fy
program_totals = rbpo_melted[['organization_id', 'program_id', 'core_resp_id', 'measure_yr', 'ftes']]
program_totals = program_totals.set_index(['organization_id', 'core_resp_id', 'measure_yr'])

program_totals

spending_fte                            program_id   ftes
organization_id core_resp_id measure_yr                  
1               BWN00        2019            BWN01  183.0
                             2020            BWN01  191.0
                             2021            BWN01  190.0
                             2022            BWN01  181.0
                             2023            BWN01  183.0
...                                            ...    ...
561             ISS00        2023            ISS00   38.0
                             2024            ISS00   32.0
                             2025            ISS00   41.0
                             2026            ISS00   41.0
                             2027            ISS00   41.0

[10875 rows x 2 columns]

In [17]:
# set up the operating costs table
op_cost['core_resp_en_clean'] = op_cost['core_responsibility_en'].apply(normalize_string)
op_cost['measure_yr'] = op_cost['fiscal_yr'].str.split('-').str[1].astype(int)

op_cost

,fiscal_yr,department_name_en,organization_id,core_responsibility_en,operating_costs,core_resp_en_clean,measure_yr
0,2022-2023,Department of Agriculture and Agri-Food,1.0,Science and Innovation,409515707,SCIENCEANDINNOVATION,2023
1,2022-2023,Department of Agriculture and Agri-Food,1.0,Domestic and International Markets,86287016,DOMESTICANDINTERNATIONALMARKETS,2023
2,2022-2023,Department of Agriculture and Agri-Food,1.0,Internal Services,239959531,INTERNALSERVICES,2023
3,2022-2023,Department of Agriculture and Agri-Food,1.0,Sector Risk,49077771,SECTORRISK,2023
4,2022-2023,Department of Agriculture and Agri-Food,1.0,Farm Credit Canada,0,FARMCREDITCANADA,2023
...,...,...,...,...,...,...,...
4844,2012-2013,Canada Border Services Agency,26.0,Internal Services,634339228,INTERNALSERVICES,2013
4845,2012-2013,Export Development Canada (Canada Account),146.0,Export Development Canada (Canada Account) - B...,6431667,EXPORTDEVELOPMENTCANADACANADAACCOUNTBUDGETARY,2013
4846,2012-2013,Export Development Canada (Canada Account),146.0,Export Development Canada (Canada Account) - N...,0,EXPORTDEVELOPMENTCANADACANADAACCOUNTNONBUDGETARY,2013
4847,2012-2013,Canadian Centre for Occupational Health and Sa...,55.0,Occupational health and safety information dev...,7037577,OCCUPATIONALHEALTHANDSAFETYINFORMATIONDEVELOPM...,2013
